# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

Provide a company name and their primary website.

In [1]:
# Import necessary libraries
import os
import requests
import json
from typing import List, Dict
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from urllib.parse import urljoin

load_dotenv()
api_key=os.getenv('DEEPSEEK_API')

if api_key and api_key[:8]=='sk-d07df':
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? ")

MODEL='deepseek-chat'
url = "https://api.deepseek.com"


openai = OpenAI(api_key=api_key, base_url=url)

class Website:
    """
    A class to represent a website and its contents.
    """
    def __init__(self, url: str):
        """
        Initialize the Website object with a given URL.
        
        :param url: The URL of the website to scrape
        """
        self.url = url
        self.title, self.text, self.links = self._scrape_website()

    def _scrape_website(self) -> tuple:
        """
        Scrape the website content, extracting title, text, and links.
        
        :return: A tuple containing the title, text content, and links of the website
        """
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.title.string if soup.title else "No title found"
        
        # Extract text content
        if soup.body:
            for tag in soup.body(["script", "style", "img", "input"]):
                tag.decompose()  # Remove unwanted tags
            text = soup.body.get_text(separator="\n", strip=True)
        else:
            text = ""
        
        # Extract links
        links = [link.get('href') for link in soup.find_all('a') if link.get('href')]
        
        return title, text, links

    def get_contents(self) -> str:
        """
        Get a formatted string of the website contents.
        
        :return: A string containing the website title and text content
        """
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

class LinkAnalyzer:
    """
    A class to analyze and categorize links from a website.
    """
    # System prompt for the OpenAI model to categorize links
    LINK_SYSTEM_PROMPT = """
    You are provided with a list of links found on a webpage. Your task is to first categorize each link into one of the following categories:
    - about page
    - careers page
    - terms of service
    - privacy policy
    - contact page
    - other (please specify).

    Once the links are categorized, please choose which links are most relevant to include in a brochure about the company. 
    The brochure should only include links such as About pages, Careers pages, or Company Overview pages. Exclude any links related to Terms of Service, Privacy Policy, or email addresses.

    Respond in the following JSON format:
    {
        "categorized_links": [
            {"category": "about page", "url": "https://full.url/about"},
            {"category": "careers page", "url": "https://full.url/careers"},
            {"category": "terms of service", "url": "https://full.url/terms"},
            {"category": "privacy policy", "url": "https://full.url/privacy"},
            {"category": "other", "specify": "contact page", "url": "https://full.url/contact"}
        ],
        "brochure_links": [
            {"type": "about page", "url": "https://full.url/about"},
            {"type": "careers page", "url": "https://full.url/careers"}
        ]
    }

    Please find the links below and proceed with the task:

    Links (some may be relative links):
    [INSERT LINK LIST HERE]
    """

    @staticmethod
    def get_links(website: Website) -> Dict:
        """
        Analyze and categorize links from a given website.
        
        :param website: A Website object containing the links to analyze
        :return: A dictionary containing categorized links and brochure-relevant links
        """
        # Prepare the user prompt for the OpenAI model
        user_prompt = f"Here is the list of links on the website of {website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(website.links)

        # Make an API call to OpenAI for link analysis
        completion = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": LinkAnalyzer.LINK_SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)

class BrochureGenerator:
    """
    A class to generate a company brochure based on website content.
    """
    # System prompt for the OpenAI model to generate the brochure
    SYSTEM_PROMPT = """
    You are an assistant that analyzes the contents of several relevant pages from a company website 
    and creates a brochure about the company for prospective customers, investors and recruits. Respond in markdown.
    Include details of company culture, customers and careers/jobs if you have the information.
    Structure the brochure to include specific sections as follows:
    About Us
    What we do
    How We Do It
    Where We Do It
    Our People
    Our Culture
    Connect with Us.
    Please provide four versions of the brochure, the first in English, the second in Japanese, the third in Chinese, the fourth in Spanish. The contents of the brochure are to be the same for both languages.
    """

    @staticmethod
    def get_all_details(url: str) -> str:
        """
        Gather all relevant details from a company's website.
        
        :param url: The URL of the company's main page
        :return: A string containing all relevant website content
        """
        result = "Landing page:\n"
        website = Website(url)
        result += website.get_contents()

        # Analyze links and get brochure-relevant ones
        links = LinkAnalyzer.get_links(website)
        brochure_links = links.get('brochure_links', [])
        print("Found Brochure links:", brochure_links)

        # Gather content from brochure-relevant pages
        for link in brochure_links:
            result += f"\n\n{link['type']}:\n"
            full_url = urljoin(url, link["url"])
            result += Website(full_url).get_contents()

        return result

    @staticmethod
    def get_brochure_user_prompt(company_name: str, url: str) -> str:
        """
        Generate a user prompt for the OpenAI model to create a brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        :return: A string containing the user prompt for brochure generation
        """
        user_prompt = f"You are looking at a company called: {company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += BrochureGenerator.get_all_details(url)
        return user_prompt[:20_000]  # Truncate if more than 20,000 characters

    @staticmethod
    def stream_brochure(company_name: str, url: str):
        """
        Generate and stream a company brochure.
        
        :param company_name: The name of the company
        :param url: The URL of the company's main page
        """
        # Make a streaming API call to OpenAI for brochure generation
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": BrochureGenerator.SYSTEM_PROMPT},
                {"role": "user", "content": BrochureGenerator.get_brochure_user_prompt(company_name, url)}
            ],
            stream=True
        )

        # Display the generated brochure in real-time
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```", "").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)


API key looks good so far


In [2]:
# Main execution block
if __name__ == "__main__":
    # Generate a brochure
    BrochureGenerator.stream_brochure("Anthropic", "https://anthropic.com")

Found Brochure links: [{'type': 'about page', 'url': 'https://anthropic.com/team'}, {'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]


# Anthropic Brochure

## About Us  
Anthropic is an AI safety and research company headquartered in San Francisco. We are dedicated to building reliable, interpretable, and steerable AI systems that prioritize safety and benefit humanity. Our interdisciplinary team includes experts in machine learning, physics, policy, and product development, working together to advance the frontier of AI research and create practical, beneficial tools like Claude.

## What We Do  
We develop cutting-edge AI models, such as Claude 3.7 Sonnet, our most intelligent model to date, and Claude Code, an agentic tool for coding. Our products are designed to assist businesses, nonprofits, and individuals in transforming how they work, from drafting documents and debugging code to creating marketing campaigns and analyzing data. We also conduct frontier research in AI safety, interpretability, and societal impacts, ensuring our systems are robustly beneficial.

## How We Do It  
We treat AI safety as a systematic science, integrating research, product development, and policy insights. Our approach is interdisciplinary, collaborative, and grounded in empirical problem-solving. We prioritize simplicity and effectiveness, focusing on solutions that deliver the greatest impact. Our commitment to transparency and responsible scaling ensures that our systems are trustworthy and secure.

## Where We Do It  
Based in San Francisco, Anthropic operates as a Public Benefit Corporation, with a mission to develop advanced AI for the long-term benefit of humanity. We collaborate with civil society, government, academia, and industry to promote safety and reliability across the AI ecosystem.

## Our People  
Our team is composed of researchers, engineers, policy experts, and operational leaders with diverse backgrounds and experiences. We value collaboration, innovation, and a shared commitment to our mission. Whether you’re a machine learning expert or new to the field, we welcome individuals who bring unique perspectives and a passion for making AI safe and beneficial.

## Our Culture  
At Anthropic, our values guide everything we do:  
1. **Act for the global good** – We prioritize long-term positive outcomes for humanity.  
2. **Hold light and shade** – We balance the risks and benefits of AI.  
3. **Be good to our users** – We cultivate kindness and generosity in all interactions.  
4. **Ignite a race to the top on safety** – We set industry standards for AI safety.  
5. **Do the simple thing that works** – We focus on practical, impactful solutions.  
6. **Be helpful, honest, and harmless** – We communicate directly and thoughtfully.  
7. **Put the mission first** – We are united by our shared purpose.  

We offer comprehensive benefits, including health and wellness support, competitive compensation, and flexible work policies, to ensure our team thrives both personally and professionally.

## Connect with Us  
Join us in shaping the future of AI. Explore our open roles, learn more about our products, or connect with us to see how Claude can transform your work.  

- **Website**: [www.anthropic.com](https://www.anthropic.com)  
- **Careers**: [Careers at Anthropic](https://www.anthropic.com/careers)  
- **Contact**: [Support Center](https://www.anthropic.com/support)  

© 2025 Anthropic PBC  

---

# アンソロピック パンフレット

## 会社概要  
アンソロピックは、サンフランシスコに本社を置くAI安全性と研究会社です。信頼性が高く、解釈可能で、操縦可能なAIシステムを構築することを使命としています。機械学習、物理学、政策、製品開発の専門家からなる学際的なチームが、AI研究の最前線を進め、Claudeのような実用的で有益なツールを開発しています。

## 私たちの活動  
私たちは、Claude 3.7 Sonnet（これまでで最も知的なモデル）やClaude Code（コーディング用のエージェントツール）などの最先端AIモデルを開発しています。これらの製品は、ビジネス、非営利団体、個人がドキュメントの作成、コードのデバッグ、マーケティングキャンペーンの作成、データ分析など、仕事の方法を変えるための支援を目的としています。また、AIの安全性、解釈可能性、社会的影響に関する最先端の研究を行い、システムが堅牢で有益であることを保証しています。

## 私たちの方法  
AIの安全性を体系的科学として扱い、研究、製品開発、政策の洞察を統合しています。私たちのアプローチは学際的で、協力的であり、実証的な問題解決に基づいています。シンプルで効果的な解決策を優先し、最大の影響を与えることに焦点を当てています。透明性と責任あるスケーリングへのコミットメントにより、信頼性と安全性を確保しています。

## 活動拠点  
サンフランシスコを拠点とするアンソロピックは、人類の長期的な利益のために高度なAIを開発することを使命とする公益法人（Public Benefit Corporation）として運営されています。市民社会、政府、学界、産業界と協力し、AIエコシステム全体の安全性と信頼性を促進しています。

## 私たちのチーム  
研究者、エンジニア、政策専門家、運営リーダーからなる私たちのチームは、多様な背景と経験を持っています。協力、革新、そして使命への共有されたコミットメントを重視しています。機械学習の専門家であろうと、この分野に初めて取り組む人であろうと、ユニークな視点とAIを安全で有益にする情熱を持つ個人を歓迎します。

## 私たちの文化  
アンソロピックでは、私たちの価値観がすべての行動を導きます：  
1. **グローバルな善のために行動する** – 人類の長期的なポジティブな結果を優先します。  
2. **光と影を抱える** – AIのリスクと利益のバランスを取ります。  
3. **ユーザーに良く接する** – すべての相互作用において親切さと寛大さを育みます。  
4. **安全性の競争を引き起こす** – AI安全性の業界基準を設定します。  
5. **シンプルで効果的なことを行う** – 実用的で影響力のある解決策に焦点を当てます。  
6. **役に立ち、正直で、害を与えない** – 直接的で思慮深いコミュニケーションを取ります。  
7. **使命を最優先する** – 私たちは共有された目的によって結ばれています。  

私たちは、健康とウェルネスのサポート、競争力のある報酬、柔軟な勤務ポリシーなど、包括的な福利厚生を提供し、チームが個人的にも職業的にも成長できるようにしています。

## お問い合わせ  
AIの未来を共に形作りましょう。私たちの求人を探す、製品について詳しく知る、またはClaudeがあなたの仕事をどのように変えるかを確認するためにご連絡ください。  

- **ウェブサイト**: [www.anthropic.com](https://www.anthropic.com)  
- **求人情報**: [アンソロピックの求人](https://www.anthropic.com/careers)  
- **お問い合わせ**: [サポートセンター](https://www.anthropic.com/support)  

© 2025 Anthropic PBC  

---

# 安思创公司手册

## 关于我们  
安思创（Anthropic）是一家位于旧金山的AI安全与研究公司。我们致力于构建可靠、可解释且可控的AI系统，以确保其对人类有益。我们的跨学科团队包括机器学习、物理学、政策和产品开发领域的专家，共同推动AI研究的前沿，并开发实用的工具，如Claude。

## 我们的工作  
我们开发尖端的AI模型，如Claude 3.7 Sonnet（我们迄今为止最智能的模型）和Claude Code（一种用于编码的代理工具）。我们的产品旨在帮助企业、非营利组织和个人改变工作方式，从起草文档和调试代码到创建营销活动和分析数据。我们还进行AI安全性、可解释性和社会影响的前沿研究，确保我们的系统具有强大的益处。

## 我们的方法  
我们将AI安全视为一门系统科学，整合研究、产品开发和政策见解。我们的方法是跨学科的、协作的，并以实证问题解决为基础。我们优先考虑简单有效的解决方案，专注于产生最大影响。我们对透明度和负责任扩展的承诺确保我们的系统值得信赖且安全。

## 我们的工作地点  
安思创总部位于旧金山，作为一家公益公司（Public Benefit Corporation），我们的使命是为人类的长期利益开发先进的AI。我们与民间社会、政府、学术界和行业合作，促进整个AI生态系统的安全性和可靠性。

## 我们的团队  
我们的团队由研究人员、工程师、政策专家和运营领导者组成，拥有多样化的背景和经验。我们重视协作、创新以及对使命的共同承诺。无论您是机器学习专家还是该领域的新手，我们都欢迎那些带来独特视角并对AI安全和有益性充满热情的个人。

## 我们的文化  
在安思创，我们的价值观指导着我们所做的一切：  
1. **为全球利益行动** – 我们优先考虑人类的长期积极成果。  
2. **平衡光与影** – 我们在AI的风险与收益之间取得平衡。  
3. **善待用户** – 我们在所有互动中培养善意和慷慨。  
4. **激发安全竞赛** – 我们为AI安全性设定行业标准。  
5. **做简单有效的事** – 我们专注于实用且有影响力的解决方案。  
6. **有帮助、诚实且无害** – 我们直接且深思熟虑地沟通。  
7. **使命至上** – 我们因共同的目标而团结一致。  

我们提供全面的福利，包括健康与健康支持、有竞争力的薪酬和灵活的工作政策，以确保我们的团队在个人和职业上都能茁壮成长。

## 联系我们  
加入我们，共同塑造AI的未来。探索我们的职位空缺，了解更多关于我们产品的信息，或与我们联系，了解Claude如何改变您的工作方式。  

- **网站**: [www.anthropic.com](https://www.anthropic.com)  
- **招聘信息**: [安思创招聘](https://www.anthropic.com/careers)  
- **联系**: [支持中心](https://www.anthropic.com/support)  

© 2025 Anthropic PBC  

---

# Folleto de Anthropic

## Sobre Nosotros  
Anthropic es una empresa de investigación y seguridad de IA con sede en San Francisco. Nos dedicamos a construir sistemas de IA confiables, interpretables y controlables que prioricen la seguridad y beneficien a la humanidad. Nuestro equipo interdisciplinario incluye expertos en aprendizaje automático, física, políticas y desarrollo de productos, trabajando juntos para avanzar en la investigación de IA y crear herramientas prácticas y beneficiosas como Claude.

## Lo Que Hacemos  
Desarrollamos modelos de IA de vanguardia, como Claude 3.7 Sonnet, nuestro modelo más inteligente hasta la fecha, y Claude Code, una herramienta agéntica para codificación. Nuestros productos están diseñados para ayudar a empresas, organizaciones sin fines de lucro e individuos a transformar su forma de trabajar, desde redactar documentos y depurar código hasta crear campañas de marketing y analizar datos. También realizamos investigaciones de vanguardia en seguridad, interpretabilidad e impactos sociales de la IA, asegurando que nuestros sistemas sean robustamente beneficiosos.

## Cómo Lo Hacemos  
Tratamos la seguridad de la IA como una ciencia sistemática, integrando investigación, desarrollo de productos y conocimientos de políticas. Nuestro enfoque es interdisciplinario, colaborativo y basado en la resolución empírica de problemas. Priorizamos soluciones simples y efectivas, enfocándonos en aquellas que generen el mayor impacto. Nuestro compromiso con la transparencia y la escalabilidad responsable asegura que nuestros sistemas sean confiables y seguros.

## Dónde Lo Hacemos  
Con sede en San Francisco, Anthropic opera como una Corporación de Beneficio Público (Public Benefit Corporation), con la misión de desarrollar IA avanzada para el beneficio a largo plazo de la humanidad. Colaboramos con la sociedad civil, gobiernos, academia e industria para promover la seguridad y confiabilidad en todo el ecosistema de IA.

## Nuestro Equipo  
Nuestro equipo está compuesto por investigadores, ingenieros, expertos en políticas y líderes operativos con diversos antecedentes y experiencias. Valoramos la colaboración, la innovación y un compromiso compartido con nuestra misión. Ya sea que seas un experto en aprendizaje automático o nuevo en el campo, damos la bienvenida a individuos que aporten perspectivas únicas y una pasión por hacer que la IA sea segura y beneficiosa.

## Nuestra Cultura  
En Anthropic, nuestros valores guían todo lo que hacemos:  
1. **Actuar por el bien global** – Priorizamos resultados positivos a largo plazo para la humanidad.  
2. **Mantener la luz y la sombra** – Equilibramos los riesgos y beneficios de la IA.  
3. **Ser buenos con nuestros usuarios** – Cultivamos la amabilidad y la generosidad en todas las interacciones.  
4. **Encender una carrera hacia la cima en seguridad** – Establecemos estándares de la industria para la seguridad de la IA.  
5. **Hacer lo simple que funciona** – Nos enfocamos en soluciones prácticas y de alto impacto.  
6. **Ser útiles, honestos e inofensivos** – Nos comunicamos de manera directa y reflexiva.  
7. **Poner la misión primero** – Estamos unidos por un propósito común.  

Ofrecemos beneficios integrales, incluyendo apoyo en salud y bienestar, compensación competitiva y políticas laborales flexibles, para asegurar que nuestro equipo prospere tanto personal como profesionalmente.

## Conéctate con Nosotros  
Únete a nosotros para dar forma al futuro de la IA. Explora nuestras vacantes, aprende más sobre nuestros productos o contáctanos para ver cómo Claude puede transformar tu trabajo.  

- **Sitio web**: [www.anthropic.com](https://www.anthropic.com)  
- **Carreras**: [Carreras en Anthropic](https://www.anthropic.com/careers)  
- **Contacto**: [Centro de Soporte](https://www.anthropic.com/support)  

© 2025 Anthropic PBC